In [43]:
import pandas as pd
import os
import warnings

pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings("ignore", message="Workbook contains no default style, apply openpyxl's default")
warnings.filterwarnings('ignore', message="A value is trying to be set on a copy of a DataFrame")

shop = 'Tiny Buds'

In [28]:
# Order Data cannot be extracted from Lazada Seller Center in one go, so we had to break it down into multiple excel files
# All excel files from the same shop are stored in one folder
folder = f'Shopee Seller Center/{shop}'
excel_files = os.listdir(folder)

# Loop over all the excel files in the folder and import only the "orderNumber" and "status" columns
dfs = {}
for file in excel_files:
    file_path = os.path.join(folder, file)
    df = pd.read_excel(file_path, usecols=["Order ID", "Order Status", "Order Creation Date"])
    dfs[file] = df

# The result will be one DataFrame with "orderNumber" and "status" columns for each excel file
for file, df in dfs.items():
    print(file)
    print("Number of rows:", len(df))

Order.all.20240316_20240406_part_10_of_14.xlsx
Number of rows: 9355
Order.all.20240316_20240406_part_11_of_14.xlsx
Number of rows: 13546
Order.all.20240316_20240406_part_12_of_14.xlsx
Number of rows: 12473
Order.all.20240316_20240406_part_13_of_14.xlsx
Number of rows: 11815
Order.all.20240316_20240406_part_14_of_14.xlsx
Number of rows: 6178
Order.all.20240316_20240406_part_1_of_14.xlsx
Number of rows: 9342
Order.all.20240316_20240406_part_2_of_14.xlsx
Number of rows: 8920
Order.all.20240316_20240406_part_3_of_14.xlsx
Number of rows: 9796
Order.all.20240316_20240406_part_4_of_14.xlsx
Number of rows: 9865
Order.all.20240316_20240406_part_5_of_14.xlsx
Number of rows: 8469
Order.all.20240316_20240406_part_6_of_14.xlsx
Number of rows: 8522
Order.all.20240316_20240406_part_7_of_14.xlsx
Number of rows: 8950
Order.all.20240316_20240406_part_8_of_14.xlsx
Number of rows: 9499
Order.all.20240316_20240406_part_9_of_14.xlsx
Number of rows: 10417


In [33]:
# Merge into one DataFrame
sc_raw = pd.concat(dfs.values(), keys=dfs.keys(), ignore_index=True)
count_sc_raw = len(sc_raw)

# Remove duplicate entries
sc_unique = sc_raw.drop_duplicates(subset=['Order ID'])
count_sc_unique = len(sc_unique)

print(f"SC Shopee {shop}")
print("Number of rows:", count_sc_raw)
print("Number of unique rows:", count_sc_unique)

SC Shopee Tiny Buds
Number of rows: 137147
Number of unique rows: 68203


In [42]:
# Import Snowflake data into a DataFrame
snowflake_raw = pd.read_csv(f'Snowflake/Snowflake Shopee {shop}.csv', usecols=['PLFM_ORD_ID','ORD_STAT_END', 'ORD_STAT_PLFM', 'ORDER_DATE'])
count_snowflake_raw = len(snowflake_raw)

# Determine date range of the dataset
date_range = pd.date_range(start=snowflake_raw.loc[:, 'ORDER_DATE'].min(), end=snowflake_raw.loc[:, 'ORDER_DATE'].max())
sf_start_date = date_range[0].date()
sf_end_date = date_range[-1].date()

# Remove duplicates
snowflake_unique = snowflake_raw.drop_duplicates(subset=['PLFM_ORD_ID'], ignore_index=True)
count_snowflake_unique = len(snowflake_unique)

# Get non-end status rows
end_status = ['delivered','cancelled','returned']

# The ~ negates the condition 'merged_unique['ORD_STAT_END'].isin(end_status)'
snowflake_non_end_status = snowflake_unique[~snowflake_unique['ORD_STAT_END'].isin(end_status)]

# Map Snowflake status
status_mapping = {
  "READY_TO_SHIP": "To ship",
  "RETRY_SHIP": "To ship",
  "PROCESSED": "To ship",
  "SHIPPED": "Shipping",
  "TO_RETURN": "Shipping",
  "TO_CONFIRM_RECEIVED": "Shipping",
  "COMPLETED": "Completed"
}

snowflake_non_end_status['ORD_STAT_PLFM'].replace(status_mapping, inplace=True)
count_snowflake_non_end_status = len(snowflake_non_end_status)

print(f"Snowflake Shopee {shop} ({sf_start_date} - {sf_end_date})")
print("Number of rows:", count_snowflake_raw)
print("Number of unique rows:", count_snowflake_unique)
print("Number of non-end status rows:", count_snowflake_non_end_status)

Snowflake Shopee Tiny Buds (2024-03-17 - 2024-04-06)
Number of rows: 160248
Number of unique rows: 65953
Number of non-end status rows: 293


C:\Users\User\AppData\Local\Temp\ipykernel_21052\451840492.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snowflake_non_end_status['ORD_STAT_PLFM'].replace(status_mapping, inplace=True)


In [40]:
# Merge the Seller Center and Snowflake DataFrames so we could compare them
merged_non_end_status = pd.merge(sc_unique, snowflake_non_end_status, left_on="Order ID", right_on="PLFM_ORD_ID", how="inner")
count_merged_non_end_status = len(merged_non_end_status)
total = count_snowflake_unique

# Count the number of mismatched status
mismatched_status = merged_non_end_status[merged_non_end_status["Order Status"] != merged_non_end_status["ORD_STAT_PLFM"]]
count_mismatched_status = len(mismatched_status)

# Calculate percent mismatched status
percent_mismatched_status = (count_mismatched_status / total) * 100

print(f"Shopee {shop} SC vs Snowflake ({sf_start_date} - {sf_end_date})")
print(mismatched_status.head(10), '\n')
print("Number of rows:", total)
print("Number of mismatched non-end status:", count_mismatched_status)
print("Percent mismatched non-end status:", round(percent_mismatched_status, 2), "%")


Shopee Tiny Buds SC vs Snowflake (2024-03-17 - 2024-04-06)
           Order ID Order Status Order Creation Date     PLFM_ORD_ID ORD_STAT_END ORD_STAT_PLFM  ORDER_DATE
205  240317A33UMVU4    Completed    2024-03-17 11:55  240317A33UMVU4      shipped      Shipping  2024-03-17
207  240317B9BVWX8R    Completed    2024-03-17 23:20  240317B9BVWX8R      shipped      Shipping  2024-03-17
209  240318D7EB403T    Completed    2024-03-18 17:51  240318D7EB403T      shipped      Shipping  2024-03-18
210  240318DTKF25U5    Completed    2024-03-18 23:34  240318DTKF25U5      shipped      Shipping  2024-03-18
219  240325VYWCW5ET    Cancelled    2024-03-25 00:19  240325VYWCW5ET       unpaid        UNPAID  2024-03-24
220  240325W17BNCWS    Cancelled    2024-03-25 00:42  240325W17BNCWS       unpaid        UNPAID  2024-03-24
221  240325W233HHRR    Cancelled    2024-03-25 00:58  240325W233HHRR       unpaid        UNPAID  2024-03-24 

Number of rows: 65953
Number of mismatched non-end status: 7
Percent mismat